In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import pathlib
DATA_DIR = pathlib.Path('/home/HDD6TB/datasets/emotions/ABAW/ABAW_7/MTL')

In [3]:
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score,roc_auc_score,average_precision_score
import mord

from collections import defaultdict
import os
import glob
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import random
import csv  
import matplotlib.pyplot as plt

In [4]:
def one_hot_transfer(label,class_num):
    return np.eye(class_num)[label]

def metric_for_Exp(gt,pred,class_num=8):
    # compute_acc
    acc = accuracy_score(gt,pred)
    # compute_F1
    gt = one_hot_transfer(gt,class_num)
    pred = one_hot_transfer(pred,class_num)
    F1 = []
    for i in range(class_num):
        gt_ = gt[:,i]
        pred_ = pred[:,i]
        F1.append(f1_score(gt_.flatten(), pred_))
    F1_mean = np.mean(F1)
    return F1_mean,acc,F1

def CCC_score(x, y):
    vx = x - np.mean(x)
    vy = y - np.mean(y)
    rho = np.sum(vx * vy) / (np.sqrt(np.sum(vx**2)) * np.sqrt(np.sum(vy**2)))
    x_m = np.mean(x)
    y_m = np.mean(y)
    x_s = np.std(x)
    y_s = np.std(y)
    ccc = 2*rho*x_s*y_s/(x_s**2 + y_s**2 + (x_m - y_m)**2)
    return ccc

def metric_for_VA(gt_V,gt_A,pred_V,pred_A):
    ccc_V,ccc_A = CCC_score(gt_V,pred_V),CCC_score(gt_A,pred_A)
    return ccc_V,ccc_A, 0.5*(ccc_V+ccc_A)

def CCC_numpy(y_true, y_pred):
    '''Reference numpy implementation of Lin's Concordance correlation coefficient'''
    
    # covariance between y_true and y_pred
    s_xy = np.cov([y_true, y_pred])[0,1]
    # means
    x_m = np.mean(y_true)
    y_m = np.mean(y_pred)
    # variances
    s_x_sq = np.var(y_true)
    s_y_sq = np.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2)
    
    return ccc

In [5]:
compare_filenames=lambda x: int(os.path.splitext(x)[0])

In [56]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout, Flatten, LSTM, Activation, MaxPooling2D
from tensorflow.keras.regularizers import l2 as L2_reg
from tensorflow.keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D, Input, Masking, Conv3D, MaxPooling3D, GlobalMaxPool3D
from tensorflow.keras.optimizers import SGD, Adam
import tensorflow.keras.backend as K 
from tensorflow.keras.models import load_model,Model

print(tf.__version__)
from tensorflow.compat.v1.keras.backend import set_session 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

2.13.1


# Feature extraction

In [6]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

print(f"Torch: {torch.__version__}")
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

Torch: 2.0.1+cu118
True


In [7]:
#num_classes=7
if False:
    #PATH='affectnet_vggface2_enet2_gmp_smooth.pt'
    PATH='enet_b2_8_best.pt'
    #PATH='enet_b2_7.pt'
    IMG_SIZE=260 #224 #
else:
    #PATH='affectnet_vggface2_enet0.pt'
    #PATH='affectnet_vggface2_enet0_new.pt'
    #PATH='enet_b0_7.pt'
    #PATH='enet_b0_8_best_afew.pt'
    #PATH='enet_b0_8_best_vgaf.pt'
    #PATH='mobilevit_va_mtl.pt'
    
    PATH='enet_b0_8_va_mtl.pt'
    IMG_SIZE=224
    
#IMG_SIZE=112

test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
np_transforms = transforms.Compose(
    [
        transforms.ToPILImage(None),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

In [7]:
if True:
    PATH='mbf_va_mtl.pt'
    
    IMG_SIZE=112
    
    test_transforms = transforms.Compose(
    [
        transforms.Resize((112,112)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])
    ]
    )
    
    import sys
    sys.path.append("/home/avsavchenko/src/distr/insightface/recognition/arcface_torch/backbones/")
    import mobilefacenet

In [8]:
print(PATH)
feature_extractor_model = torch.load('/home/avsavchenko/src/face-emotion-recognition/models/affectnet_emotions/'+PATH)

enet_b0_8_va_mtl.pt


In [7]:
if True:
    IMG_SIZE=112
    
    test_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])  
    
    import sys
    DDAMNFN_PATH='/home/avsavchenko/src/distr/DDAMFN'
    sys.path.append(DDAMNFN_PATH)
    from networks.DDAM import DDAMNet

    if False:
        feature_extractor_model = DDAMNet(num_class=8, num_head=2)
        model_name='affectnet8_epoch4_acc0.6462'
    else:
        feature_extractor_model = DDAMNet(num_class=10, num_head=2)
        model_name='affectnet8_epoch9_acc0.642_mtl_2'
    feature_extractor_model.load_state_dict(torch.load(DDAMNFN_PATH+'/checkpoints/'+model_name+'.pth')['model_state_dict'])
    feature_extractor_model.bn=torch.nn.Identity()

In [8]:
if False:
    classifier_weights=feature_extractor_model.classifier[0].weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier[0].bias.cpu().data.numpy()
elif False:
    classifier_weights=feature_extractor_model.classifier.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier.bias.cpu().data.numpy()
elif False:
    classifier_weights=feature_extractor_model.head.fc.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.head.fc.bias.cpu().data.numpy()
else:
    classifier_weights=feature_extractor_model.fc.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.fc.bias.cpu().data.numpy()
print(classifier_weights.shape,classifier_weights)
print(classifier_bias.shape,classifier_bias)

(10, 512) [[ 0.01794354 -0.03431091 -0.00016818 ... -0.0268224  -0.02347743
   0.02176051]
 [-0.02611435 -0.02214147  0.03573207 ... -0.00466314  0.04239551
  -0.01990408]
 [-0.05963856 -0.01730311  0.04917104 ... -0.01539477 -0.00746309
  -0.01494454]
 ...
 [ 0.04475736 -0.00453137  0.0154746  ... -0.00426385 -0.00999709
  -0.01408496]
 [ 0.0183069  -0.00611857 -0.00720661 ...  0.00141637 -0.01378473
  -0.00045164]
 [ 0.01398357  0.00257701  0.00391316 ... -0.00689993 -0.0030514
   0.03322613]]
(10,) [ 0.00227628  0.00355737 -0.00111303 -0.03025234 -0.0463375   0.0778885
  0.01492943 -0.01847218 -0.05123357  0.05881068]


In [9]:
if False:
    feature_extractor_model.classifier=torch.nn.Identity()
elif False:
    feature_extractor_model.head.fc=torch.nn.Identity()
else:
    feature_extractor_model.fc=torch.nn.Identity()
feature_extractor_model=feature_extractor_model.to(device)
feature_extractor_model.eval()

DDAMNet(
  (features): Sequential(
    (0): Conv_block(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
    )
    (1): Conv_block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
    )
    (2): Mix_Depth_Wise(
      (conv): Conv_block(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (prelu): PReLU(num_parameters=128)
      )
      (conv_dw): MDConv(
        (mixed_depthwise_conv): ModuleList(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
    

In [10]:
def get_probab(features, logits=True):
    x=np.dot(features,np.transpose(classifier_weights))+classifier_bias
    if logits:
        return x
    #print(x)
    e_x = np.exp(x - np.max(x,axis=1)[:,np.newaxis])
    #print(x.shape,x,np.max(x,axis=1),e_x)
    return e_x / e_x.sum(axis=1)[:,None]

In [12]:
print(test_transforms)
#data_dir=DATA_DIR / 'cropped'
data_dir=DATA_DIR / 'cropped_aligned'
print(data_dir)

img_names=[]
X_global_features=[]
imgs=[]
for filename in tqdm(os.listdir(data_dir)):
    frames_dir=data_dir / filename 
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img = Image.open(data_dir /filename / img_name)
            img_tensor = test_transforms(img)
            if img.size:
                img_names.append(filename+'/'+img_name)
                imgs.append(img_tensor)
                if len(imgs)>=48: #64: #96: #32:        
                    #features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features,_,_  = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    #print(features.shape)

                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)
                    imgs=[]

if len(imgs)>0:        
    #features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features,_,_  = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features=features.data.cpu().numpy()

    if len(X_global_features)==0:
        X_global_features=features
    else:
        X_global_features=np.concatenate((X_global_features,features),axis=0)

    imgs=[]

    #X_scores=X_global_features #get_probab(X_global_features)
    #print(X_global_features.shape,X_scores.shape)

Compose(
    Resize(size=(112, 112), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/home/HDD6TB/datasets/emotions/ABAW/ABAW_7/MTL/cropped_aligned


  0%|          | 0/307 [00:00<?, ?it/s]

In [13]:
X_scores=get_probab(X_global_features,logits=True)
print(X_scores[:5])

[[-0.5149662  -1.5605701  -3.2025433   0.60394394 -1.3945719   3.668983
   1.9120594   3.0826316   0.1800033  -0.00792246]
 [-0.46407315 -1.5230858  -0.25438294  0.99677694 -0.9453583   1.9377406
   0.02826373  2.1803274  -0.09846481  0.3727368 ]
 [-0.37511447  0.5664461  -0.01685829 -1.8778757   0.39609146  2.349856
  -0.48270184  0.26101306  0.06444739  0.09477985]
 [-0.58261037 -1.288324    0.19343507 -0.47934464  2.1812882   0.90803945
  -0.7950299   1.3629999   0.53139037  0.21014062]
 [-1.1705606  -1.4704839   1.3758544   0.44316047 -0.63199896  1.0219592
   0.5419112   0.9813946  -0.2273141   0.32000583]]


In [14]:
filename2featuresAll={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
#filename2featuresAll={img_name:(global_features,) for img_name,global_features, in zip(img_names,X_global_features)}
print(len(filename2featuresAll))

168097


In [13]:
img_names=list(filename2featuresAll.keys())
print(len(img_names))
X_global_features=np.array([gf for gf,_ in filename2featuresAll.values()])
print(X_global_features.shape)
print(img_names[0],X_global_features[0],filename2featuresAll[img_names[0]])

168097
(168097, 1280)
116/01399.jpg [-0.19524716  0.04839505 -0.02437067 ... -0.02058871 -0.01529629
 -0.10724838] (array([-0.19524716,  0.04839505, -0.02437067, ..., -0.02058871,
       -0.01529629, -0.10724838], dtype=float32), array([ 1.7540247 , -2.0302808 , -2.8912916 ,  0.29071587, -1.2524096 ,
        3.0524902 ,  0.16354522,  3.6558971 , -0.11794583,  0.18994373],
      dtype=float32))


## Save/load features 

In [57]:
import pickle
num_classes=8

#model_name='enet_b2_8_best'
#model_name='enet_b0_8_best_vgaf'
#model_name='mobilevit_mtl'
#model_name='mbf_va'

#model_name='ddamfnet_8' #affectnet8_epoch4_acc0.6462
model_name='ddamfnet_8_mtl' #affectnet8_epoch6_acc0

#model_name='enet_b0_8_va_mtl_1'
#model_name='enet_b0_8_va_mtl'

if True:
    model_name+='_final'

#model_name+='_cropped'#
model_name+='_aligned'#
#model_name+='_super_cropped'
#model_name+='_super_aligned'
#model_name+='_super_SR_aligned'

MODEL2FEATURES=model_name+'.pickle' 

print(MODEL2FEATURES)

ddamfnet_8_mtl_final_aligned.pickle


In [17]:
if False:
    with open(MODEL2FEATURES, 'wb') as handle:
        pickle.dump(filename2featuresAll, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [58]:
with open(MODEL2FEATURES, 'rb') as handle:
    filename2featuresAll=pickle.load(handle)
print(len(filename2featuresAll))

168097


## Test set 

In [11]:
print(test_transforms)
data_dir=DATA_DIR / 'test_data/cropped_aligned'
print(data_dir)
img_names=[]
X_global_features=[]
imgs=[]
for filename in tqdm(os.listdir(data_dir)):
    frames_dir=data_dir / filename 
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img = Image.open(data_dir /filename / img_name)
            img_tensor = test_transforms(img)
            if img.size:
                img_names.append(filename+'/'+img_name)
                imgs.append(img_tensor)
                if len(imgs)>=48: #64: #96: #32:        
                    #features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features,_,_  = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    #print(features.shape)

                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)
                    imgs=[]

if len(imgs)>0:        
    #features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features,_,_  = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features=features.data.cpu().numpy()

    if len(X_global_features)==0:
        X_global_features=features
    else:
        X_global_features=np.concatenate((X_global_features,features),axis=0)

    imgs=[]


X_scores=get_probab(X_global_features,logits=True)
print(X_scores[:5])
filename2featuresTest={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
print(len(filename2featuresTest))

Compose(
    Resize(size=(112, 112), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/home/HDD6TB/datasets/emotions/ABAW/ABAW_7/MTL/test_data/cropped_aligned


  0%|          | 0/82 [00:00<?, ?it/s]

[[-0.5986035  -1.0780891  -1.6025386   0.23478708  1.8031008   0.55298734
  -0.9814955   2.3338978   0.40912086  0.488054  ]
 [-0.64827454 -0.44641376 -0.87043446 -1.1551245  -1.2319028   1.8593757
   1.0840331   1.9555519  -0.07612106  0.05349598]
 [-0.54896116 -3.0227926  -2.1910653   2.8041844  -0.21186872  1.4450716
  -0.7420143   4.0839868   0.27756202  0.8472906 ]
 [-0.48197263 -0.3958168  -0.35849625 -2.9457254   0.8756877   2.5106077
   1.568931    0.46133715 -0.11000779 -0.14062907]
 [-0.7725098   1.3602316  -1.7234982  -1.9215229   2.140337    1.6366392
  -1.5881624   0.7132013   0.29523742  0.14544609]]
50441


In [137]:
MODEL2FEATURES_TEST=model_name+'_test.pickle' 
print(MODEL2FEATURES_TEST)

enet_b0_8_va_mtl_final_aligned_test.pickle


In [14]:
with open(MODEL2FEATURES_TEST, 'wb') as handle:
    pickle.dump(filename2featuresTest, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [138]:
with open(MODEL2FEATURES_TEST, 'rb') as handle:
    filename2featuresTest=pickle.load(handle)
print(len(filename2featuresTest))

50441


# Classifiers

In [59]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.applications import efficientnet as enet
import efficientnet.tfkeras as enet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [60]:
from copy import deepcopy
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())
                
def CCC(y_true, y_pred):
    '''Lin's Concordance correlation coefficient: https://en.wikipedia.org/wiki/Concordance_correlation_coefficient
    
    The concordance correlation coefficient is the correlation between two variables that fall on the 45 degree line through the origin.
    
    It is a product of
    - precision (Pearson correlation coefficient) and
    - accuracy (closeness to 45 degree line)

    Interpretation:
    - `rho_c =  1` : perfect agreement
    - `rho_c =  0` : no agreement
    - `rho_c = -1` : perfect disagreement 
    
    Args: 
    - y_true: ground truth
    - y_pred: predicted values
    
    Returns:
    - concordance correlation coefficient (float)
    '''
    
    
    # covariance between y_true and y_pred
    #N = K.int_shape(y_pred)[-1]
    #s_xy = 1.0 / (N - 1.0 + K.epsilon()) * K.sum((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    #s_xy = K.mean(K.sum((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred))))
    s_xy = K.mean((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    # means
    x_m = K.mean(y_true)
    y_m = K.mean(y_pred)
    # variances
    s_x_sq = K.var(y_true)
    s_y_sq = K.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2+K.epsilon())
    #print(s_xy,s_x_sq,s_y_sq,x_m,y_m)
    return ccc

def CCC_VA(y_true, y_pred):
    return 1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))


import math
def f1_score_max_for_AU_one_class(gt, pred, thresh,type=0):
    gt = gt[:,type]
    pred = pred[:,type]
    P = []
    R = []
    ACC = []
    F1 = []
    for i in thresh:
        new_pred = ((pred >= i) * 1).flatten()
        P.append(precision_score(gt.flatten(), new_pred))
        R.append(recall_score(gt.flatten(), new_pred))
        ACC.append(accuracy_score(gt.flatten(), new_pred))
        F1.append(f1_score(gt.flatten(), new_pred))

    F1_MAX = max(F1)
    if F1_MAX < 0 or math.isnan(F1_MAX):
        F1_MAX = 0
        F1_THRESH = 0
        accuracy = 0
    else:
        idx_thresh = np.argmax(F1)
        F1_THRESH = thresh[idx_thresh]
        accuracy = ACC[idx_thresh]
    return F1,F1_MAX,F1_THRESH,accuracy

def f1_score_max(gt, pred, thresh,c=12):
    F1_s = []
    F1_t = []
    ACC = []
    from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
    for i in range(c):
        F1, F1_MAX, F1_THRESH,acc = f1_score_max_for_AU_one_class(gt,pred,thresh,i)
        F1_s.append(F1_MAX)
        F1_t.append(F1_THRESH)
        ACC.append(acc)
    F1_s=np.array(F1_s)
    F1_t=np.array(F1_t)
    ACC=np.array(ACC)
    return F1_s.mean(),F1_t.mean(),F1_s,F1_t,ACC

## Multi-task challenge 

In [61]:
FEATURES_ONLY,SCORES_ONLY,FEATURES_SCORES=0,1,2
FEATURES_SCORES_AGGREGATION=FEATURES_SCORES #FEATURES_ONLY #
SCORES_LEN=8
def get_image2all(filename):
    global SCORES_LEN
    with open(DATA_DIR / filename) as f:
        mtl_lines = f.read().splitlines()
    num_missed=0
    X,y_va,y_expr,y_aus=[],[],[],[]
    masks_va,masks_expr,masks_aus=[],[],[]
    videoname_frames=[]
    for line in mtl_lines[1:]:
        splitted_line=line.split(',')
        imagename=splitted_line[0]
        valence=float(splitted_line[1])
        arousal=float(splitted_line[2])
        expression=int(splitted_line[3])
        aus=list(map(int,splitted_line[4:]))
        
        if False:
            imagename=imagename[:-4]+'_out.jpg'
            #print(imagename)
        
        mask_VA=(valence>-5 and arousal>-5)
        if not mask_VA:
            valence=arousal=0
            
        mask_expr=(expression>-1)
        if not mask_expr:
            expression=0
            
        mask_aus=min(aus)>=0
        if not mask_aus:
            aus=[0]*len(aus)
        #print(imagename,valence,arousal,expression,aus)
        if mask_VA or mask_expr or mask_aus:
            if imagename in filename2featuresAll:
                videoname,filename=imagename.split('/')
                frame_ind=int(filename.replace('.jpg',''))
                videoname_frames.append((videoname,frame_ind))
                
                SCORES_LEN=len(filename2featuresAll[imagename][1])
                if FEATURES_SCORES_AGGREGATION==FEATURES_ONLY:
                    X.append(filename2featuresAll[imagename][0])
                elif FEATURES_SCORES_AGGREGATION==SCORES_ONLY:
                    X.append(filename2featuresAll[imagename][1])
                else:
                    X.append(np.concatenate((filename2featuresAll[imagename][0],filename2featuresAll[imagename][1])))
                y_va.append((valence,arousal))
                masks_va.append(mask_VA)
                
                y_expr.append(expression)
                masks_expr.append(mask_expr)
                
                y_aus.append(aus)
                masks_aus.append(mask_aus)
            else:
                num_missed+=1
        #elif (valence==-5 or arousal==-5) and valence!=arousal:
        #    print(valence,arousal)
    X=np.array(X)
    y_va=np.array(y_va)
    y_expr=np.array(y_expr)
    y_aus=np.array(y_aus)
    masks_va=np.array(masks_va).astype(np.float32)
    masks_expr=np.array(masks_expr).astype(np.float32)
    masks_aus=np.array(masks_aus).astype(np.float32)
    print(X.shape,y_va.shape,y_expr.shape,y_aus.shape,masks_va.shape,num_missed)
    return X,y_va,y_expr,y_aus,masks_va,masks_expr,masks_aus,videoname_frames

X_train,y_va_train,y_expr_train,y_aus_train,masks_va_train,masks_expr_train,masks_aus_train,_=get_image2all('training_set_annotations.txt')
X_val,y_va_val,y_expr_val,y_aus_val,masks_va_val,masks_expr_val,masks_aus_val,videoname_frames_val=get_image2all('validation_set_annotations.txt')
TRAIN_VAL=False
print(SCORES_LEN)
#cropped: (142333, 1288) (142333, 2) (142333,) (142333, 12) (142333,) 0
#(26876, 1288) (26876, 2) (26876,) (26876, 12) (26876,) 0

(142333, 522) (142333, 2) (142333,) (142333, 12) (142333,) 0
(26876, 522) (26876, 2) (26876,) (26876, 12) (26876,) 0
10


In [140]:
if True:
    TRAIN_VAL=True
    X_train=np.concatenate((X_train,X_val))
    y_va_train=np.concatenate((y_va_train,y_va_val))
    y_expr_train=np.concatenate((y_expr_train,y_expr_val))
    y_aus_train=np.concatenate((y_aus_train,y_aus_val))
    masks_va_train=np.concatenate((masks_va_train,masks_va_val))
    masks_expr_train=np.concatenate((masks_expr_train,masks_expr_val))
    masks_aus_train=np.concatenate((masks_aus_train,masks_aus_val))
    print(X_train.shape,y_va_train.shape,y_expr_train.shape,y_aus_train.shape,
          masks_va_train.shape,masks_expr_train.shape,masks_aus_train.shape)

(169209, 1290) (169209, 2) (169209,) (169209, 12) (169209,) (169209,) (169209,)


In [62]:
def print_expr():
    y_val_preds,_,_=mtl_model_new.predict(X_val,verbose=0)
    y_pred=np.argmax(y_val_preds,axis=1)[masks_expr_val==1]
    print('Acc:',(y_pred==y_expr_only_val).mean(), 'F1:',f1_score(y_true=y_expr_only_val,y_pred=y_pred, average="macro"))
    print(metric_for_Exp(y_expr_only_val,y_pred))

def print_ccc():
    _,y_val_preds,_=mtl_model_new.predict(X_val,verbose=0)
    pred_V=y_val_preds[masks_va_val==1,0]
    pred_A=y_val_preds[masks_va_val==1,1]
    gt_V=y_va_only_val[:,0]
    gt_A=y_va_only_val[:,1]
    print(metric_for_VA(gt_V,gt_A,pred_V,pred_A))
    print(CCC_numpy(gt_V,pred_V),CCC_numpy(gt_A,pred_A))

def print_au():
    _,_,y_val_preds=mtl_model_new.predict(X_val,verbose=0)
    y_val_preds=y_val_preds[masks_va_val==1,:]
    new_pred = ((y_val_preds >= 0.5) * 1)
    print(np.mean([f1_score(y_true=y_au_only_val[:,i],y_pred=new_pred[:,i]) for i in range(y_val_preds.shape[1])]))
    print(f1_score_max(y_au_only_val,y_val_preds,thresh=np.arange(0.1,1,0.1)))

In [63]:
print(np.unique(y_expr_val[masks_expr_val==1.].astype(int), return_counts=True))

(unique, counts) = np.unique(y_expr_train[masks_expr_train==1.].astype(int), return_counts=True)
num_classes=len(unique)
emo_cw=1/counts
#emo_cw*=counts.mean()
emo_cw/=emo_cw.min()
emo_class_weights = {i:cwi for i,cwi in zip(unique,emo_cw)}
print(counts, emo_class_weights, num_classes, unique)
print(emo_cw)
if False:
    emo_cw=np.array([1,5,5,5,1,1,1])

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([1886,  487,  565, 1254, 3751, 1893, 1003, 4601]))
[23976  4555  3168  3122 18135  7609  5228 24852] {0: 1.0365365365365367, 1: 5.455982436882547, 2: 7.844696969696971, 3: 7.960281870595772, 4: 1.3703887510339123, 5: 3.266132211854383, 6: 4.7536342769701605, 7: 1.0} 8 [0 1 2 3 4 5 6 7]
[1.03653654 5.45598244 7.84469697 7.96028187 1.37038875 3.26613221
 4.75363428 1.        ]


In [64]:
num_labels=y_aus_train.shape[1]
print(num_labels)
from sklearn.utils.class_weight import compute_class_weight
aus_class_weights = np.empty([num_labels, 2])
for i in range(num_labels):
    neg, pos = np.bincount(y_aus_train[masks_aus_train==1., i])
    print(i,neg,pos,np.bincount(y_aus_val[masks_aus_val==1., i]))
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    aus_class_weights[i][0]=weight_for_0
    aus_class_weights[i][1]=weight_for_1
    #aus_class_weights[i] = compute_class_weight('balanced', [0,1], y_train[:, i])
print(aus_class_weights)

12
0 84603 18713 [21345  5531]
1 94571 8745 [24322  2554]
2 82974 20342 [21434  5442]
3 72711 30605 [18058  8818]
4 61979 41337 [13652 13224]
5 65030 38286 [15455 11421]
6 76533 26783 [18557  8319]
7 100856 2460 [26094   782]
8 100336 2980 [26119   757]
9 98313 5003 [26068   808]
10 32732 70584 [ 6449 20427]
11 92560 10756 [23942  2934]
[[ 0.610593    2.7605408 ]
 [ 0.5462351   5.90714694]
 [ 0.62258057  2.53947498]
 [ 0.71045646  1.68789413]
 [ 0.83347585  1.24967946]
 [ 0.79437183  1.34926605]
 [ 0.67497681  1.92876078]
 [ 0.51219561 20.99918699]
 [ 0.5148501  17.33489933]
 [ 0.52544424 10.32540476]
 [ 1.57821093  0.73186558]
 [ 0.55810285  4.80271476]]


In [65]:
def loss_va(y_true, y_pred):
    res=1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))
    #res=K.mean((y_true[:,0] - y_pred[:,0]) * (y_true[:,0] - y_pred[:,0]))+K.mean((y_true[:,1] - y_pred[:,1]) * (y_true[:,1] - y_pred[:,1]))
    #print(res)
    return res

In [67]:
if True:
    def get_weighted_loss_aus(weights):
        def weighted_loss(y_true, y_pred):
            y_true=tf.cast(y_true, tf.float32)
            ce=K.binary_crossentropy(y_true, y_pred)
            res=K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*ce)#, axis=-1)
            #print(res)
            return res
        return weighted_loss
    loss_aus=get_weighted_loss_aus(aus_class_weights)
else:
    pos_weights=np.array([1, 2, 1, 1, 1, 1, 1, 6, 6, 5, 1, 5])
    def loss_aus(y_true, y_pred):
        y_true=tf.cast(y_true, tf.float32)
        ce=K.binary_crossentropy(y_true, y_pred)
        res=K.mean((pos_weights**(y_true))*ce)#, axis=-1)
        #print(res)
        return res

In [68]:
metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()] # 

## MTL model

In [69]:
X_expr_only_train,y_expr_only_train=X_train[masks_expr_train==1],y_expr_train[masks_expr_train==1]
X_expr_only_val,y_expr_only_val=X_val[masks_expr_val==1],y_expr_val[masks_expr_val==1]
print(X_expr_only_train.shape,y_expr_only_train.shape,X_expr_only_val.shape,y_expr_only_val.shape)

X_va_only_train,y_va_only_train=X_train[masks_va_train==1],y_va_train[masks_va_train==1]
X_va_only_val,y_va_only_val=X_val[masks_va_val==1],y_va_val[masks_va_val==1]
print(X_va_only_train.shape,y_va_only_train.shape,X_va_only_val.shape,y_va_only_val.shape)

X_au_only_train,y_au_only_train=X_train[masks_aus_train==1],y_aus_train[masks_aus_train==1]
X_au_only_val,y_au_only_val=X_val[masks_aus_val==1],y_aus_val[masks_aus_val==1]
print(X_au_only_train.shape,y_au_only_train.shape,X_au_only_val.shape,y_au_only_val.shape)

(90645, 522) (90645,) (15440, 522) (15440,)
(103917, 522) (103917, 2) (26876, 522) (26876, 2)
(103316, 522) (103316, 12) (26876, 522) (26876, 12)


aligned
(90645, 522) (90645,) (15440, 522) (15440,)
(103917, 522) (103917, 2) (26876, 522) (26876, 2)
(103316, 522) (103316, 12) (26876, 522) (26876, 12)

In [70]:
class Slice(tf.keras.layers.Layer):
    def __init__(self, begin, size,**kwargs):
        super(Slice, self).__init__(**kwargs)
        self.begin = begin
        self.size = size
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'begin': self.begin,
            'size': self.size,
        })
        return config
    def call(self, inputs):
        #return tf.slice(inputs, self.begin, self.size)
        return inputs[:,self.begin:self.begin+self.size]

In [71]:
batch_size=512
img = tf.keras.Input(shape=X_train.shape[1:])
x=img
#x=Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
if True:
    expr_out=Dense(8, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
else:
    s_l=Slice(0,X_train.shape[1]-SCORES_LEN)(x)
    expr_out=Dense(8, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(s_l)
if False:
    va_out = Dense(2, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
else:
    s_l=Slice(X_train.shape[1]-SCORES_LEN,SCORES_LEN)(x)
    va_out = Dense(2, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(s_l)
    
if False:
    aus_out=Dense(12, activation='sigmoid',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
else:
    au_fc=Dense(128,activation='relu')(x)
    aus_out=Dense(12, activation='sigmoid')(au_fc)

mtl_model_new=tf.keras.Model(inputs=img, outputs=[expr_out, va_out, aus_out])

In [40]:
import copy
if True:
    initial_weights=copy.deepcopy(mtl_model_new.get_weights())
else:
    mtl_model_new.set_weights(copy.deepcopy(initial_weights))

### Expression

In [143]:
#mtl_model_new.set_weights(copy.deepcopy(initial_weights))
model_expr=tf.keras.Model(inputs=img, outputs=expr_out)
model_expr.compile(optimizer=Adam(lr=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_expr.summary()

save_best_model = SaveBestModel('val_accuracy',True)
model_expr.fit(X_expr_only_train,y_expr_only_train, batch_size=batch_size, epochs=(1 if TRAIN_VAL else 10), verbose=1, 
             callbacks=[save_best_model], validation_data=(X_expr_only_val,y_expr_only_val),class_weight=emo_class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1290)]            0         
                                                                 
 dense_16 (Dense)            (None, 8)                 10328     
                                                                 
Total params: 10328 (40.34 KB)
Trainable params: 10328 (40.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
208/208 [==============================] - 2s 6ms/step - loss: 2.8629 - accuracy: 0.5364 - val_loss: 1.4370 - val_accuracy: 0.5205
0.5204663276672363


In [42]:
if True:
    print_expr()
    print('Best weights:')
    model_expr.set_weights(best_model_weights)
print_expr()

Acc: 0.29481865284974096 F1: 0.2496361065661037
(0.2496361065661037, 0.29481865284974096, [0.295774647887324, 0.2286689419795222, 0.18072289156626506, 0.10005053057099546, 0.4413657990484187, 0.26945126945126946, 0.18265682656826568, 0.298397945456769])
Best weights:
Acc: 0.31023316062176165 F1: 0.2863993761001457
(0.2863993761001457, 0.31023316062176165, [0.3358878913947887, 0.2332242225859247, 0.28638867033831633, 0.1427406199021207, 0.4285041594726103, 0.2996542451018056, 0.25740944017563117, 0.30738575982996813])


### Valence-arousal 

In [144]:
model_va=tf.keras.Model(inputs=img, outputs=va_out)
model_va.compile(optimizer=Adam(lr=1e-3), loss=CCC_VA, metrics=['mae','mse'])
model_va.summary()

save_best_model = SaveBestModel('val_loss',False)
model_va.fit(X_va_only_train,y_va_only_train, batch_size=batch_size, epochs=(10 if TRAIN_VAL else 20), verbose=1, callbacks=[save_best_model], 
          validation_data=(X_va_only_val,y_va_only_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1290)]            0         
                                                                 
 slice_4 (Slice)             (None, 10)                0         
                                                                 
 dense_17 (Dense)            (None, 2)                 22        
                                                                 
Total params: 22 (88.00 Byte)
Trainable params: 22 (88.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
256/256 [==============================] - 3s 8ms/step - loss: 0.9099 - mae: 0.8025 - mse: 0.8812 - val_loss: 0.8981 - val_mae: 0.7198 - val_mse: 0.7492
Epoch 2/10
256/256 [==============================] - 2s 7ms/step - loss: 0.7362 - mae: 0.6258 - mse: 0.5701 - val_loss: 0.80

In [44]:
if True:
    print_ccc()
    print('Best weights:')
    model_va.set_weights(best_model_weights)
print_ccc()

(0.4768937702188683, 0.4170483231056068, 0.4469710466622375)
0.47691150678167504 0.4170638481070598
Best weights:
(0.47615733862577536, 0.41967165510380056, 0.44791449686478796)
0.47617503740781003 0.4196872808166141


### Action Units 

In [145]:
#mtl_model_new.set_weights(copy.deepcopy(initial_weights))
model_au=tf.keras.Model(inputs=img, outputs=aus_out)
model_au.compile(optimizer=Adam(lr=1e-3), loss=loss_aus, metrics=metrics)
model_au.summary()

save_best_model = SaveBestModel('val_loss',False)
model_au.fit(X_au_only_train,y_au_only_train, batch_size=batch_size, epochs=10, verbose=1, callbacks=[save_best_model], 
          validation_data=(X_au_only_val,y_au_only_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1290)]            0         
                                                                 
 dense_18 (Dense)            (None, 128)               165248    
                                                                 
 dense_19 (Dense)            (None, 12)                1548      
                                                                 
Total params: 166796 (651.55 KB)
Trainable params: 166796 (651.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
255/255 [==============================] - 3s 8ms/step - loss: 0.4309 - auc: 0.8927 - binary_accuracy: 0.8144 - recall_1: 0.7785 - precision_1: 0.5747 - val_loss: 0.4197 - val_auc: 0.8895 - val_binary_accuracy: 0.8133 - val_recall_1: 0.7858 - val_precision_1: 0.5976
Epoc

In [46]:
print_au()

0.48943836635800003
(0.5065387658887107, 0.5750000000000001, array([0.596875  , 0.42337263, 0.58607418, 0.5744446 , 0.74169654,
       0.71253207, 0.66858963, 0.21157851, 0.16619605, 0.16112676,
       0.87694644, 0.35903279]), array([0.7, 0.8, 0.5, 0.3, 0.4, 0.4, 0.6, 0.7, 0.8, 0.8, 0.2, 0.7]), array([0.83680607, 0.87442328, 0.82393213, 0.67000298, 0.71844769,
       0.72901473, 0.79479833, 0.92956541, 0.92309123, 0.94459741,
       0.79799821, 0.81161631]))


In [75]:
print('Best weights:')
model_au.set_weights(best_model_weights)
print_au()

Best weights:
0.48907253367367104
(0.5056922470403562, 0.5666666666666668, array([0.57338664, 0.42434783, 0.60371812, 0.59266419, 0.73246152,
       0.7077644 , 0.67308648, 0.17116902, 0.14770459, 0.20616659,
       0.87543221, 0.36040539]), array([0.6, 0.7, 0.5, 0.4, 0.4, 0.5, 0.6, 0.6, 0.8, 0.8, 0.2, 0.7]), array([0.8054398 , 0.87684179, 0.82947611, 0.7020762 , 0.7076946 ,
       0.73083792, 0.80421194, 0.87100015, 0.95233666, 0.93581634,
       0.79624944, 0.82858312]))


### Combine all 

In [72]:
MODEL_WEIGHTS=model_name.replace('_final','')+'.h5'
print(MODEL_WEIGHTS)

ddamfnet_8_mtl_aligned.h5


In [73]:
if False:
    mtl_model_new.save_weights(MODEL_WEIGHTS)
else:
    mtl_model_new.load_weights(MODEL_WEIGHTS)

In [74]:
y_pred_expr,y_pred_va,y_pred_aus=mtl_model_new.predict(X_val,verbose=0)
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
pred_V=y_pred_va[masks_va_val==1,0]
pred_A=y_pred_va[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(gt_V.shape,ccc_V,ccc_A,ccc_VA)

print('\nExpression')
print(y_expr_val[masks_expr_val==1].shape)
y_pred=np.argmax(y_pred_expr,axis=1)
print((y_pred==y_expr_val)[masks_expr_val==1].mean())
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
print(f1_expr)
print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))

print('\nAUs')
new_pred = ((y_pred_aus >= 0.5) * 1)
print(new_pred[masks_aus_val==1,:].shape)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au,f1_per_au)
print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)


VA
(26876,) 0.49261329654719144 0.47251579687472106 0.4825645467109563

Expression
(15440,)
0.3642487046632124
0.3396390388473775
(0.3396390388473775, 0.3642487046632124, [0.3949496942197672, 0.21012416427889208, 0.30106007067137813, 0.2596613113330438, 0.4796486090775988, 0.5042708003796266, 0.2638888888888889, 0.30350877192982456])

AUs
(26876, 12)
0.49061325563161157 [0.5948654691234292, 0.38266431407146007, 0.5932950669661425, 0.5851615036930878, 0.7153840355769957, 0.7118462299950504, 0.6846329603374539, 0.18832087290361688, 0.10728582866293035, 0.15306795689465583, 0.8205971114464324, 0.35023771790808245]
(0.5154035097723907, 0.5583333333333332, array([0.61233933, 0.43602466, 0.59603997, 0.5876745 , 0.72167842,
       0.71184623, 0.68463296, 0.22790698, 0.16476965, 0.19921105,
       0.88059534, 0.36212304]), array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]), array([0.83167138, 0.8604331 , 0.83755023, 0.68458848, 0.69742521,
       0.74006549, 0.79970978, 0.925

aligned
enet0_mtl_1 0.4433236497636661 0.3422118665954431 0.504019732383684 1.2895552487427933
#enet0_mtl 0.44008861133398225 0.3276690856479878 0.49110471073031176 1.2588624077122819
mbf_va 0.4502576708835253 0.28702860556472376 0.4891132466734737 1.2263995231217228
mobilevit 0.437905348251858 0.31455641686900754 0.48171682380243847 1.2341785889233041
ddamfn 0.46906593486826226 0.3148428907106757 0.49443618408892237 1.2783450096678604
#ddamfn_mtl 0.4844231828965665 0.32227849609132764 0.4897985168087399 1.296500195796634
ddamfn_mtl 0.4825645467109563 0.3396390388473775 0.49061325563161157 1.3128168411899455

cropped
enet0_mtl_1 0.44686599794668075 0.3308134303417443 0.49426554663277256 1.2719449749211975
#enet0_mtl 0.4470512345513264 0.33507035885799447 0.4943377041019861 1.276459297511307
mbf_va 0.42674369168984627 0.2783961649368074 0.4731084221837445 1.1782482788103983
ddamfn 0.4505480142418006 0.30454545345220785 0.4772574389449172 1.2323509066389255
ddamfn_mtl 0.44791449686478796 0.2863993761001457 0.48943836635800003 1.2237522393229336


super cropped
enet0_mtl 0.4445623273123246 0.33855943145262635 0.4958321228309311 1.278953881595882
ddamfn_mtl 0.4532791059616273 0.29621963252768 0.4843917557414951 1.2338904942308024

super aligned
enet0_mtl 0.4341307250465519 0.32185679066511785 0.48744505429137935 1.243432570003049


cropped SR super
enet0_mtl 0.4098072453440361 0.28090699515169476 0.49178146929494626 1.1824957097906772

## Smoothing 

In [35]:
y_pred_expr,y_pred_va,y_pred_aus=mtl_model_new.predict(X_val,verbose=0)
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
pred_V=y_pred_va[masks_va_val==1,0]
pred_A=y_pred_va[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(gt_V.shape,ccc_V,ccc_A,ccc_VA)

print('\nExpression')
print(y_expr_val[masks_expr_val==1].shape)
y_pred=np.argmax(y_pred_expr,axis=1)
print((y_pred==y_expr_val)[masks_expr_val==1].mean())
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
print(f1_expr)
print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))

print('\nAUs')
new_pred = ((y_pred_aus >= 0.5) * 1)
print(new_pred[masks_aus_val==1,:].shape)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au,f1_per_au)
print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)


VA
(26876,) 0.49261329654719144 0.47251579687472106 0.4825645467109563

Expression
(15440,)
0.3642487046632124
0.3396390388473775
(0.3396390388473775, 0.3642487046632124, [0.3949496942197672, 0.21012416427889208, 0.30106007067137813, 0.2596613113330438, 0.4796486090775988, 0.5042708003796266, 0.2638888888888889, 0.30350877192982456])

AUs
(26876, 12)
0.49061325563161157 [0.5948654691234292, 0.38266431407146007, 0.5932950669661425, 0.5851615036930878, 0.7153840355769957, 0.7118462299950504, 0.6846329603374539, 0.18832087290361688, 0.10728582866293035, 0.15306795689465583, 0.8205971114464324, 0.35023771790808245]
(0.5154035097723907, 0.5583333333333332, array([0.61233933, 0.43602466, 0.59603997, 0.5876745 , 0.72167842,
       0.71184623, 0.68463296, 0.22790698, 0.16476965, 0.19921105,
       0.88059534, 0.36212304]), array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]), array([0.83167138, 0.8604331 , 0.83755023, 0.68458848, 0.69742521,
       0.74006549, 0.79970978, 0.925

In [36]:
thresholds=np.array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]) #ddamfn_mtl
#thresholds=np.array([0.5, 0.8, 0.5, 0.4, 0.4, 0.4, 0.6, 0.7, 0.8, 0.6, 0.2, 0.6]) #enet0_mtl
new_pred = ((y_pred_aus >= thresholds) * 1)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au,f1_per_au)

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)

0.5154035097723907 [0.6123393316195372, 0.4360246579461735, 0.5960399703923019, 0.5876744977868573, 0.7216784174139229, 0.7118462299950504, 0.6846329603374539, 0.22790697674418606, 0.16476964769647698, 0.1992110453648915, 0.8805953421648398, 0.3621230398069964]

Total 0.4825645467109563 0.3396390388473775 0.5154035097723907 1.3376070953307244


In [37]:
import math

print('Expr')
#delta=1
#sigma=1.0
num_y=len(y_pred_expr)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,5,10,50,100]:
        y_pred_expr_smooth=np.zeros_like(y_pred_expr)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_expr_smooth[i]+=k*y_pred_expr[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_expr_smooth[i]+=k*y_pred_expr[j]
                    k_sum+=k
                else:
                    break
            y_pred_expr_smooth[i]/=k_sum

        y_pred=np.argmax(y_pred_expr_smooth,axis=1)
        #print((y_pred==y_expr_val)[masks_expr_val==1].mean())
        f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
        print(sigma,delta,f1_expr)
    print()

print('VA')
num_y=len(y_pred_va)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,5,10,50,100]:
        y_pred_va_smooth=np.zeros_like(y_pred_va)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_va_smooth[i]+=k*y_pred_va[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_va_smooth[i]+=k*y_pred_va[j]
                    k_sum+=k
                else:
                    break
            y_pred_va_smooth[i]/=k_sum

        pred_V=y_pred_va_smooth[masks_va_val==1,0]
        pred_A=y_pred_va_smooth[masks_va_val==1,1]
        ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
        print(sigma,delta,ccc_V,ccc_A,ccc_VA)
    print()

print('AU')
num_y=len(y_pred_aus)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,2,5,10]:
        y_pred_aus_smooth=np.zeros_like(y_pred_aus)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_aus_smooth[i]+=k*y_pred_aus[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_aus_smooth[i]+=k*y_pred_aus[j]
                    k_sum+=k
                else:
                    break
            y_pred_va_smooth[i]/=k_sum

        new_pred = ((y_pred_aus_smooth >= 0.5) * 1)
        f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
        f1_au=np.mean(f1_per_au)
        print(sigma,delta,f1_au)
    print()

Expr
0.1 1 0.3396390388473775
0.1 5 0.3396390388473775
0.1 10 0.3396390388473775
0.1 50 0.3396390388473775
0.1 100 0.3396390388473775

1 1 0.33964568083873825
1 5 0.34016832669266106
1 10 0.34016832669266106
1 50 0.34016832669266106
1 100 0.34016832669266106

10 1 0.3398635994486794
10 5 0.3430931763986831
10 10 0.34314306468912403
10 50 0.34314306468912403
10 100 0.34314306468912403

50 1 0.34381331246508123
50 5 0.3534128435044339
50 10 0.3540113317082747
50 50 0.3540113317082747
50 100 0.3540113317082747

100 1 0.3439591943141603
100 5 0.35705401519282165
100 10 0.35880834427621955
100 50 0.3589404812166007
100 100 0.3589404812166007

500 1 0.34535692622707814
500 5 0.3692966413891762
500 10 0.3718829462264696
500 50 0.37296675210362973
500 100 0.37296675210362973

1000 1 0.34557326390059584
1000 5 0.37166649946984853
1000 10 0.380014532772065
1000 50 0.3829728752144952
1000 100 0.3829728752144952

10000 1 0.34580944113198686
10000 5 0.3745547587828839
10000 10 0.38586904679926626
1

In [38]:
y_pred_expr,y_pred_va,y_pred_aus=mtl_model_new.predict(X_val,verbose=0)
print('Expr')
sigma,delta=100000,50 #ddamfn
#sigma,delta=10000,100 #enet0_mtl
y_pred_expr_smooth=np.zeros_like(y_pred_expr)
num_y=len(y_pred_expr)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr[j]
            k_sum+=k
        else:
            break
    y_pred_expr_smooth[i]/=k_sum

y_pred=np.argmax(y_pred_expr_smooth,axis=1)
#print((y_pred==y_expr_val)[masks_expr_val==1].mean())
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
print(f1_expr)

print('\nVA')
sigma,delta=100000,10
y_pred_va_smooth=np.zeros_like(y_pred_va)
num_y=len(y_pred_va)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_va_smooth[i]+=k*y_pred_va[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_va_smooth[i]+=k*y_pred_va[j]
            k_sum+=k
        else:
            break
    y_pred_va_smooth[i]/=k_sum

pred_V=y_pred_va_smooth[masks_va_val==1,0]
pred_A=y_pred_va_smooth[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(sigma,delta,ccc_V,ccc_A,ccc_VA)
    
print('\nAUs')
new_pred = ((y_pred_aus >= 0.5) * 1)
print(new_pred[masks_aus_val==1,:].shape)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au,f1_per_au)
print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)

Expr
0.40506163067713824

VA
100000 10 0.5603400970051672 0.5597089899703704 0.5600245434877689

AUs
(26876, 12)
0.49061325563161157 [0.5948654691234292, 0.38266431407146007, 0.5932950669661425, 0.5851615036930878, 0.7153840355769957, 0.7118462299950504, 0.6846329603374539, 0.18832087290361688, 0.10728582866293035, 0.15306795689465583, 0.8205971114464324, 0.35023771790808245]
(0.5154035097723907, 0.5583333333333332, array([0.61233933, 0.43602466, 0.59603997, 0.5876745 , 0.72167842,
       0.71184623, 0.68463296, 0.22790698, 0.16476965, 0.19921105,
       0.88059534, 0.36212304]), array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]), array([0.83167138, 0.8604331 , 0.83755023, 0.68458848, 0.69742521,
       0.74006549, 0.79970978, 0.92588183, 0.9426626 , 0.93957434,
       0.80179342, 0.80324453]))

Total 0.5600245434877689 0.40506163067713824 0.49061325563161157 1.4556994297965187


In [39]:
thresholds=np.array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]) #ddamfn_mtl
#thresholds=np.array([0.5, 0.8, 0.5, 0.4, 0.4, 0.4, 0.6, 0.7, 0.8, 0.6, 0.2, 0.6]) #enet0_mtl
new_pred = ((y_pred_aus >= thresholds) * 1)
print(new_pred[masks_aus_val==1,:].shape)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au,f1_per_au)

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)

(26876, 12)
0.5154035097723907 [0.6123393316195372, 0.4360246579461735, 0.5960399703923019, 0.5876744977868573, 0.7216784174139229, 0.7118462299950504, 0.6846329603374539, 0.22790697674418606, 0.16476964769647698, 0.1992110453648915, 0.8805953421648398, 0.3621230398069964]

Total 0.5600245434877689 0.40506163067713824 0.5154035097723907 1.4804896839372979


In [75]:
sigma,delta=100000,50 #ddamfn
#sigma,delta=10000,100 #enet0_mtl
y_pred_expr_smooth=np.zeros_like(y_pred_expr)
num_y=len(y_pred_expr)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr[j]
            k_sum+=k
        else:
            break
    y_pred_expr_smooth[i]/=k_sum

y_pred_smooth=np.argmax(y_pred_expr_smooth,axis=1)
print((y_pred==y_expr_val)[masks_expr_val==1].mean(),(y_pred_smooth==y_expr_val)[masks_expr_val==1].mean())
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred_smooth[masks_expr_val==1], average="macro")
print(f1_expr)


0.3642487046632124 0.4265544041450777
0.40506163067713824


-------------

In [25]:
#y_pred_expr_1,y_pred_va_1,y_pred_aus_1=y_pred_expr,y_pred_va,y_pred_aus
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
pred_V_1=y_pred_va_1[masks_va_val==1,0]
pred_A_1=y_pred_va_1[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V_1,pred_A_1)
print(ccc_V,ccc_A,ccc_VA)

print('\nExpression')
y_pred_1=np.argmax(y_pred_expr_1,axis=1)
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred_1[masks_expr_val==1], average="macro")
print(f1_expr)

print('\nAUs')
new_pred_1 = ((y_pred_aus_1 >= 0.5) * 1)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred_1[masks_aus_val==1,i]) for i in range(y_pred_aus_1.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au)

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)

print('Expr')
num_y=len(y_pred_expr_1)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,5,10,50,100]:
        y_pred_expr_smooth=np.zeros_like(y_pred_expr_1)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_expr_smooth[i]+=k*y_pred_expr_1[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_expr_smooth[i]+=k*y_pred_expr_1[j]
                    k_sum+=k
                else:
                    break
            y_pred_expr_smooth[i]/=k_sum

        y_pred=np.argmax(y_pred_expr_smooth,axis=1)
        #print((y_pred==y_expr_val)[masks_expr_val==1].mean())
        f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
        print(sigma,delta,f1_expr)
    print()

print('\nVA')
num_y=len(y_pred_va_1)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,5,10,50,100]:
        y_pred_va_smooth=np.zeros_like(y_pred_va_1)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_va_smooth[i]+=k*y_pred_va_1[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_va_smooth[i]+=k*y_pred_va_1[j]
                    k_sum+=k
                else:
                    break
            y_pred_va_smooth[i]/=k_sum

        pred_V=y_pred_va_smooth[masks_va_val==1,0]
        pred_A=y_pred_va_smooth[masks_va_val==1,1]
        ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
        print(sigma,delta,ccc_V,ccc_A,ccc_VA)
    print()
    
print('\nAU')
num_y=len(y_pred_aus_1)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,2,5,10]:
        y_pred_aus_smooth=np.zeros_like(y_pred_aus_1)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_aus_smooth[i]+=k*y_pred_aus_1[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_aus_smooth[i]+=k*y_pred_aus_1[j]
                    k_sum+=k
                else:
                    break
            y_pred_va_smooth[i]/=k_sum

        new_pred = ((y_pred_aus_smooth >= 0.5) * 1)
        f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
        f1_au=np.mean(f1_per_au)
        print(sigma,delta,f1_au)
    print()


VA
0.4663397082341116 0.42030759129322065 0.4433236497636661

Expression
0.3422118665954431

AUs
0.504019732383684

Total 0.4433236497636661 0.3422118665954431 0.504019732383684 1.2895552487427933
Expr
0.1 1 0.3422118665954431
0.1 5 0.3422118665954431
0.1 10 0.3422118665954431
0.1 50 0.3422118665954431
0.1 100 0.3422118665954431

1 1 0.3428655506488497
1 5 0.34355104472438425
1 10 0.34355104472438425
1 50 0.34355104472438425
1 100 0.34355104472438425

10 1 0.3452579047384535
10 5 0.3469648342004372
10 10 0.34700239467708033
10 50 0.34700239467708033
10 100 0.34700239467708033

50 1 0.34988998966529283
50 5 0.3558411519560144
50 10 0.35572462989805936
50 50 0.3556808913480353
50 100 0.3556808913480353

100 1 0.3497123312145829
100 5 0.3606439860951177
100 10 0.36065311901544717
100 50 0.360890327391747
100 100 0.360890327391747

500 1 0.3520862891256603
500 5 0.36725072638804923
500 10 0.3676344507784245
500 50 0.36892740042657357
500 100 0.36892740042657357

1000 1 0.35251161972145695

## Ensemble 

In [49]:
y_pred_expr_1,y_pred_va_1,y_pred_aus_1=y_pred_expr,y_pred_va,y_pred_aus
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
pred_V_1=y_pred_va_1[masks_va_val==1,0]
pred_A_1=y_pred_va_1[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V_1,pred_A_1)
print(ccc_V,ccc_A,ccc_VA)

print('\nExpression')
y_pred_1=np.argmax(y_pred_expr_1,axis=1)
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred_1[masks_expr_val==1], average="macro")
print(f1_expr)

print('\nAUs')
new_pred_1 = ((y_pred_aus_1 >= 0.5) * 1)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred_1[masks_aus_val==1,i]) for i in range(y_pred_aus_1.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au)

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)


VA
0.4663397082341116 0.42030759129322065 0.4433236497636661

Expression
0.3422118665954431

AUs
0.504019732383684

Total 0.4433236497636661 0.3422118665954431 0.504019732383684 1.2895552487427933


In [100]:
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
pred_V=y_pred_va[masks_va_val==1,0]
pred_A=y_pred_va[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(gt_V.shape,ccc_V,ccc_A,ccc_VA)

print('\nExpression')
print(y_expr_val[masks_expr_val==1].shape)
y_pred=np.argmax(y_pred_expr,axis=1)
print((y_pred==y_expr_val)[masks_expr_val==1].mean())
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
print(f1_expr)
print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))

print('\nAUs')
new_pred = ((y_pred_aus >= 0.5) * 1)
print(new_pred[masks_aus_val==1,:].shape)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au,f1_per_au)
print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)


VA
(26876,) 0.49261329654719144 0.47251579687472106 0.4825645467109563

Expression
(15440,)
0.3642487046632124
0.3396390388473775
(0.3396390388473775, 0.3642487046632124, [0.3949496942197672, 0.21012416427889208, 0.30106007067137813, 0.2596613113330438, 0.4796486090775988, 0.5042708003796266, 0.2638888888888889, 0.30350877192982456])

AUs
(26876, 12)
0.49061325563161157 [0.5948654691234292, 0.38266431407146007, 0.5932950669661425, 0.5851615036930878, 0.7153840355769957, 0.7118462299950504, 0.6846329603374539, 0.18832087290361688, 0.10728582866293035, 0.15306795689465583, 0.8205971114464324, 0.35023771790808245]
(0.5154035097723907, 0.5583333333333332, array([0.61233933, 0.43602466, 0.59603997, 0.5876745 , 0.72167842,
       0.71184623, 0.68463296, 0.22790698, 0.16476965, 0.19921105,
       0.88059534, 0.36212304]), array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]), array([0.83167138, 0.8604331 , 0.83755023, 0.68458848, 0.69742521,
       0.74006549, 0.79970978, 0.925

In [38]:
pred_V=y_pred_va[masks_va_val==1,0]
pred_A=y_pred_va[masks_va_val==1,1]
for w_blend in np.linspace(0,1,11):
    pred_V_blend=w_blend*pred_V+(1-w_blend)*pred_V_1
    pred_A_blend=w_blend*pred_A+(1-w_blend)*pred_A_1
    ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V_blend,pred_A_blend)
    print(w_blend,ccc_V,ccc_A,ccc_VA)

0.0 0.4663397082341116 0.42030759129322065 0.4433236497636661
0.1 0.47414693571058086 0.43110913625841424 0.4526280359844975
0.2 0.48089890826583015 0.44083676882410067 0.4608678385449654
0.30000000000000004 0.4865310319386979 0.44939250762768623 0.46796176978319204
0.4 0.4909938891225386 0.4566965998940376 0.47384524450828813
0.5 0.49425490004471917 0.46269016076388286 0.47847253040430104
0.6000000000000001 0.4962991029948709 0.4673367751320353 0.4818179390634531
0.7000000000000001 0.49712914076539194 0.4706229154018089 0.4838760280836004
0.8 0.49676504494109636 0.47255773547436747 0.4846613902077319
0.9 0.4952428448955219 0.4731720526696966 0.48420744878260924
1.0 0.49261329654719144 0.47251579687472106 0.4825645467109563


In [39]:
for w_blend in np.linspace(0,1,11):
    y_pred_expr_blend=w_blend*y_pred_expr+(1-w_blend)*y_pred_expr_1
    y_pred=np.argmax(y_pred_expr_blend,axis=1)
    f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
    print(w_blend,f1_expr)


0.0 0.3422118665954431
0.1 0.3484338524443187
0.2 0.3534822372452379
0.30000000000000004 0.35725657829092955
0.4 0.35679560346279937
0.5 0.3578294980076967
0.6000000000000001 0.35611129683266596
0.7000000000000001 0.3532055613190705
0.8 0.34803534265745767
0.9 0.3417128144487849
1.0 0.3396390388473775


In [40]:
for w_blend in np.linspace(0,1,11):
    y_pred_aus_blend=w_blend*y_pred_aus+(1-w_blend)*y_pred_aus_1
    new_pred = ((y_pred_aus_blend >= 0.5) * 1)
    f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus_blend.shape[1])]
    f1_au=np.mean(f1_per_au)
    print(w_blend,f1_au)


0.0 0.504019732383684
0.1 0.5082943613315123
0.2 0.5118645491586519
0.30000000000000004 0.515145533556098
0.4 0.51734097145394
0.5 0.5193726515139269
0.6000000000000001 0.5192203614426717
0.7000000000000001 0.513637047587021
0.8 0.507063182220946
0.9 0.49882651230612746
1.0 0.49061325563161157


---------------

In [50]:
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
w_blend=0.8
y_pred_va_blend=w_blend*y_pred_va+(1-w_blend)*y_pred_va_1
pred_V=y_pred_va_blend[masks_va_val==1,0]
pred_A=y_pred_va_blend[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(ccc_V,ccc_A,ccc_VA)

print('\nExpression')
w_blend=0.5
y_pred_expr_blend=w_blend*y_pred_expr+(1-w_blend)*y_pred_expr_1
y_pred_blend=np.argmax(y_pred_expr_blend,axis=1)
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred_blend[masks_expr_val==1], average="macro")
print(f1_expr)
print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred_blend[masks_expr_val==1]))


print('\nAUs')
w_blend=0.5
y_pred_aus_blend=w_blend*y_pred_aus+(1-w_blend)*y_pred_aus_1
new_pred = ((y_pred_aus_blend >= 0.5) * 1)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus_blend.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au)
print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))


total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)


VA
0.49676504494109636 0.47255773547436747 0.4846613902077319

Expression
0.3578294980076967
(0.3578294980076967, 0.37441709844559584, [0.39449917898193754, 0.21520146520146521, 0.39918809201623817, 0.2620689655172414, 0.49876111354030034, 0.5157024793388431, 0.2677304964539007, 0.30948419301164726])

AUs
0.5193726515139269
(0.5154035097723907, 0.5583333333333332, array([0.61233933, 0.43602466, 0.59603997, 0.5876745 , 0.72167842,
       0.71184623, 0.68463296, 0.22790698, 0.16476965, 0.19921105,
       0.88059534, 0.36212304]), array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]), array([0.83167138, 0.8604331 , 0.83755023, 0.68458848, 0.69742521,
       0.74006549, 0.79970978, 0.92588183, 0.9426626 , 0.93957434,
       0.80179342, 0.80324453]))

Total 0.4846613902077319 0.3578294980076967 0.5193726515139269 1.3618635397293555


In [105]:
print('Expr')
num_y=len(y_pred_expr_blend)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,5,10,50,100]:
        y_pred_expr_smooth=np.zeros_like(y_pred_expr_blend)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_expr_smooth[i]+=k*y_pred_expr_blend[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_expr_smooth[i]+=k*y_pred_expr_blend[j]
                    k_sum+=k
                else:
                    break
            y_pred_expr_smooth[i]/=k_sum

        y_pred=np.argmax(y_pred_expr_smooth,axis=1)
        #print((y_pred==y_expr_val)[masks_expr_val==1].mean())
        f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
        print(sigma,delta,f1_expr)
    print()

print('\nVA')
num_y=len(y_pred_va_blend)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,5,10,50,100]:
        y_pred_va_smooth=np.zeros_like(y_pred_va_blend)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_va_smooth[i]+=k*y_pred_va_blend[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_va_smooth[i]+=k*y_pred_va_blend[j]
                    k_sum+=k
                else:
                    break
            y_pred_va_smooth[i]/=k_sum

        pred_V=y_pred_va_smooth[masks_va_val==1,0]
        pred_A=y_pred_va_smooth[masks_va_val==1,1]
        ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
        print(sigma,delta,ccc_V,ccc_A,ccc_VA)
    print()
    
print('\nAU')
num_y=len(y_pred_aus_blend)
for sigma in [0.1, 1, 10, 50, 100, 500, 1000, 10000, 100000]:
    for delta in [1,2,5,10]:
        y_pred_aus_smooth=np.zeros_like(y_pred_aus_blend)
        for i in range(num_y):
            start,end=max(0,i-delta),min(num_y,i+delta+1)
            k_sum=0

            for j in range(i-1,start,-1):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_aus_smooth[i]+=k*y_pred_aus_blend[j]
                    k_sum+=k
                else:
                    break
            for j in range(i,end):
                if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
                    k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
                    y_pred_aus_smooth[i]+=k*y_pred_aus_blend[j]
                    k_sum+=k
                else:
                    break
            y_pred_va_smooth[i]/=k_sum

        new_pred = ((y_pred_aus_smooth >= 0.5) * 1)
        f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
        f1_au=np.mean(f1_per_au)
        print(sigma,delta,f1_au)
    print()

Expr
0.1 1 0.3578294980076967
0.1 5 0.3578294980076967
0.1 10 0.3578294980076967
0.1 50 0.3578294980076967
0.1 100 0.3578294980076967

1 1 0.3580040769840508
1 5 0.35787379706275946
1 10 0.35787379706275946
1 50 0.35787379706275946
1 100 0.35787379706275946

10 1 0.3595589599688823
10 5 0.3618704379508414
10 10 0.36191247228776535
10 50 0.36191247228776535
10 100 0.36191247228776535

50 1 0.36461966795046014
50 5 0.37087090968172154
50 10 0.37167578257206824
50 50 0.37167578257206824
50 100 0.37167578257206824

100 1 0.36723069406595316
100 5 0.37676955681235635
100 10 0.37677506708541936
100 50 0.3768586031443939
100 100 0.3768586031443939

500 1 0.367572759686546
500 5 0.38747710275197617
500 10 0.38870202919439245
500 50 0.389154523822414
500 100 0.389154523822414

1000 1 0.3678265454104591
1000 5 0.3901531487086407
1000 10 0.39318932393225514
1000 50 0.3951129013780777
1000 100 0.3951129013780777

10000 1 0.36758848459974275
10000 5 0.3929480805984306
10000 10 0.40142351535030857
1

In [47]:
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
w_blend=0.8
y_pred_va_blend=w_blend*y_pred_va+(1-w_blend)*y_pred_va_1
num_y=len(y_pred_va_blend)
sigma,delta=100000,10
y_pred_va_smooth=np.zeros_like(y_pred_va_blend)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_va_smooth[i]+=k*y_pred_va_blend[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_va_smooth[i]+=k*y_pred_va_blend[j]
            k_sum+=k
        else:
            break
    y_pred_va_smooth[i]/=k_sum

pred_V=y_pred_va_smooth[masks_va_val==1,0]
pred_A=y_pred_va_smooth[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(ccc_V,ccc_A,ccc_VA)

print('\nExpression')
w_blend=0.5
y_pred_expr_blend=w_blend*y_pred_expr+(1-w_blend)*y_pred_expr_1
num_y=len(y_pred_expr_blend)
sigma,delta=100000,50
y_pred_expr_smooth=np.zeros_like(y_pred_expr_blend)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr_blend[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_val[j][0]==videoname_frames_val[i][0]:
            k=math.exp(-(videoname_frames_val[j][1]-videoname_frames_val[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr_blend[j]
            k_sum+=k
        else:
            break
    y_pred_expr_smooth[i]/=k_sum

y_pred=np.argmax(y_pred_expr_smooth,axis=1)
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
print(f1_expr)
print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))

print('\nAUs')
w_blend=0.5
y_pred_aus_blend=w_blend*y_pred_aus+(1-w_blend)*y_pred_aus_1
new_pred = ((y_pred_aus_blend >= 0.5) * 1)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus_blend.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au,f1_per_au)
print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))


total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)


VA
0.5597742989662853 0.5557893615003944 0.5577818302333398

Expression
Expr
0.41675297202366557
(0.41675297202366557, 0.4280440414507772, [0.43381725021349277, 0.22683264177040113, 0.5172932330827068, 0.30252873563218385, 0.513066082911479, 0.7331900521952717, 0.24413145539906106, 0.3631643249847282])

AUs
0.5193726515139269 [0.6173096976016684, 0.4400950650076891, 0.5924099674209738, 0.607473508087005, 0.7388817819249003, 0.7225979918497668, 0.6884361549497848, 0.2575024201355276, 0.16606843742522134, 0.18736842105263157, 0.8474833311543993, 0.36684504155755443]
(0.5154035097723907, 0.5583333333333332, array([0.61233933, 0.43602466, 0.59603997, 0.5876745 , 0.72167842,
       0.71184623, 0.68463296, 0.22790698, 0.16476965, 0.19921105,
       0.88059534, 0.36212304]), array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]), array([0.83167138, 0.8604331 , 0.83755023, 0.68458848, 0.69742521,
       0.74006549, 0.79970978, 0.92588183, 0.9426626 , 0.93957434,
       0.80179342

# Save test results

In [6]:
with open(os.path.join(DATA_DIR,'../test_set_examples/MTL.txt'),'r') as f:
    test_set_images=f.read().splitlines()
print(len(test_set_images),test_set_images[:5])

51160 ['image,valence,arousal,expression,aus', 'video45_1/00001.jpg,', 'video45_1/00008.jpg,', 'video45_1/00017.jpg,', 'video45_1/00021.jpg,']


In [7]:
resdir=DATA_DIR /'test_results/MTL'
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'image,valence,arousal,expression,aus\n'

In [147]:
num_missed=0
X_test=[]
for imagename in test_set_images[1:]:
    if imagename in filename2featuresTest:
        if FEATURES_SCORES_AGGREGATION==FEATURES_ONLY:
            X_test.append(filename2featuresTest[imagename][0])
        elif FEATURES_SCORES_AGGREGATION==SCORES_ONLY:
            X_test.append(filename2featuresTest[imagename][1])
        else:
            X_test.append(np.concatenate((filename2featuresTest[imagename][0],filename2featuresTest[imagename][1])))
    else:
        num_missed+=1
    #elif (valence==-5 or arousal==-5) and valence!=arousal:
    #    print(valence,arousal)
X_test=np.array(X_test)
print(X_test.shape,num_missed)

(51159, 1290) 0


In [45]:
videoname_frames_test=[]
for imagename in test_set_images[1:]:
    if imagename in filename2featuresTest:
        videoname,filename=imagename.split('/')
        frame_ind=int(filename.replace('.jpg',''))
        videoname_frames_test.append((videoname,frame_ind))
print(len(videoname_frames_test),videoname_frames_test[:10])

51159 [('video45_1', 1), ('video45_1', 8), ('video45_1', 17), ('video45_1', 21), ('video45_1', 77), ('video45_1', 79), ('video45_1', 84), ('video45_1', 85), ('video45_1', 88), ('video45_1', 90)]


In [148]:
y_pred_expr,y_pred_va,y_pred_aus=mtl_model_new.predict(X_test,verbose=0)

In [149]:
pred_V=y_pred_va[:,0]
pred_A=y_pred_va[:,1]
print('VA',pred_V.shape,pred_A.shape,pred_V[:5],pred_A[:5])

y_pred=np.argmax(y_pred_expr,axis=1)
print('Expression',y_pred_expr.shape,y_pred.shape,y_pred[:5])

thresholds=0.5
aus_pred = ((y_pred_aus >= thresholds) * 1)
print('AUs',y_pred_aus.shape,aus_pred.shape,aus_pred[:5])

#thresholds=np.array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]) #ddamfn_mtl
thresholds=np.array([0.5, 0.8, 0.5, 0.4, 0.4, 0.4, 0.6, 0.7, 0.8, 0.6, 0.2, 0.6]) #enet0_mtl
aus_pred_dynamic = ((y_pred_aus >= thresholds) * 1)
print('AUs',y_pred_aus.shape,aus_pred_dynamic.shape,aus_pred_dynamic[:5])

VA (51159,) (51159,) [0.6993264  0.48666823 0.5255217  0.47365773 0.39880493] [0.3263365  0.367703   0.3712185  0.43587178 0.552522  ]
Expression (51159, 8) (51159,) [7 5 7 4 7]
AUs (51159, 12) (51159, 12) [[0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 0 0 1 0]]
AUs (51159, 12) (51159, 12) [[0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 0 0 1 0]]


In [150]:
#with open(os.path.join(resdir,'1_mtl_ddamfn_aus_0.5.txt'), 'w') as f:
#with open(os.path.join(resdir,'3_mtl_enet0_aus_0.5.txt'), 'w') as f:
#with open(os.path.join(resdir,'7_mtl_ddamfn_train_val_aus_0.5.txt'), 'w') as f:
with open(os.path.join(resdir,'9_mtl_enet0_train_val_aus_0.5.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)

In [151]:
#with open(os.path.join(resdir,'1_mtl_ddamfn_aus_dyn.txt'), 'w') as f:
#with open(os.path.join(resdir,'3_mtl_enet0_aus_dyn.txt'), 'w') as f:
#with open(os.path.join(resdir,'7_mtl_ddamfn_train_val_aus_dyn.txt'), 'w') as f:
with open(os.path.join(resdir,'9_mtl_enet0_train_val_aus_dyn.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred_dynamic):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)

In [163]:
#y_pred_expr,y_pred_va,y_pred_aus=y_pred_expr_test,y_pred_va_test,y_pred_aus_test

In [164]:
print('Expr')
sigma,delta=100000,50 #ddamfn
#sigma,delta=10000,100 #enet0_mtl
y_pred_expr_smooth=np.zeros_like(y_pred_expr)
num_y=len(y_pred_expr)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_test[j][0]==videoname_frames_test[i][0]:
            k=math.exp(-(videoname_frames_test[j][1]-videoname_frames_test[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_test[j][0]==videoname_frames_test[i][0]:
            k=math.exp(-(videoname_frames_test[j][1]-videoname_frames_test[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr[j]
            k_sum+=k
        else:
            break
    y_pred_expr_smooth[i]/=k_sum

y_pred=np.argmax(y_pred_expr_smooth,axis=1)
print('Expression',y_expr_val.shape,y_pred.shape,y_pred[:5])

print('\nVA')
sigma,delta=100000,10
y_pred_va_smooth=np.zeros_like(y_pred_va)
num_y=len(y_pred_va)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_test[j][0]==videoname_frames_test[i][0]:
            k=math.exp(-(videoname_frames_test[j][1]-videoname_frames_test[i][1])**2/sigma)
            y_pred_va_smooth[i]+=k*y_pred_va[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_test[j][0]==videoname_frames_test[i][0]:
            k=math.exp(-(videoname_frames_test[j][1]-videoname_frames_test[i][1])**2/sigma)
            y_pred_va_smooth[i]+=k*y_pred_va[j]
            k_sum+=k
        else:
            break
    y_pred_va_smooth[i]/=k_sum

pred_V=y_pred_va_smooth[:,0]
pred_A=y_pred_va_smooth[:,1]
print('VA',pred_V.shape,pred_A.shape,pred_V[:5],pred_A[:5])
    
thresholds=0.5
aus_pred = ((y_pred_aus >= thresholds) * 1)
print('AUs',y_pred_aus.shape,aus_pred.shape,aus_pred[:5])

thresholds=np.array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.7, 0.8, 0.8, 0.1, 0.6]) #ddamfn_mtl
#thresholds=np.array([0.5, 0.8, 0.5, 0.4, 0.4, 0.4, 0.6, 0.7, 0.8, 0.6, 0.2, 0.6]) #enet0_mtl
aus_pred_dynamic = ((y_pred_aus >= thresholds) * 1)
print('AUs',y_pred_aus.shape,aus_pred_dynamic.shape,aus_pred_dynamic[:5])

Expr
Expression (26876,) (51159,) [4 4 4 4 4]

VA
VA (51159,) (51159,) [0.39632756 0.41739503 0.4453415  0.4720601  0.50465363] [0.57142293 0.5694437  0.57116747 0.57154447 0.5685369 ]
AUs (51159, 12) (51159, 12) [[0 0 0 0 1 1 1 0 0 0 1 1]
 [1 1 0 0 1 1 1 0 0 0 1 1]
 [0 0 1 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]]
AUs (51159, 12) (51159, 12) [[0 0 0 0 1 1 1 0 0 0 1 1]
 [1 0 0 1 1 1 1 0 0 0 1 1]
 [0 0 1 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]]


In [165]:
#with open(os.path.join(resdir,'2_mtl_ddamfn_smooth_aus_0.5.txt'), 'w') as f:
#with open(os.path.join(resdir,'4_mtl_enet0_smooth_aus_0.5.txt'), 'w') as f:
#with open(os.path.join(resdir,'8_mtl_enet0_smooth_train_val_aus_0.5.txt'), 'w') as f:
with open(os.path.join(resdir,'10_mtl_ddamfn_smooth_train_val_aus_0.5.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)

In [166]:
#with open(os.path.join(resdir,'2_mtl_ddamfn_smooth_aus_dyn.txt'), 'w') as f:
#with open(os.path.join(resdir,'4_mtl_enet0_smooth_aus_dyn.txt'), 'w') as f:
#with open(os.path.join(resdir,'8_mtl_enet0_smooth_train_val_aus_dyn.txt'), 'w') as f:
with open(os.path.join(resdir,'10_mtl_ddamfn_smooth_train_val_aus_dyn.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred_dynamic):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)

In [162]:
print(y_pred_expr_test,y_pred_expr)
(y_pred_expr_test==y_pred_expr).mean()

[[0.0630198  0.02923042 0.01153559 ... 0.12221114 0.1366621  0.20354486]
 [0.07535907 0.01654283 0.01829018 ... 0.07680057 0.07374471 0.3088546 ]
 [0.04144539 0.05796424 0.09958213 ... 0.1374438  0.13033669 0.28298432]
 ...
 [0.05700237 0.03572528 0.08230273 ... 0.0011791  0.00788018 0.1097265 ]
 [0.02096938 0.05481327 0.18771867 ... 0.00191849 0.00620825 0.09658289]
 [0.03223616 0.05434929 0.06769163 ... 0.00132441 0.02141548 0.44234413]] [[0.0754565  0.03244578 0.03053937 ... 0.1281305  0.19092736 0.32821357]
 [0.08677309 0.02282087 0.03861262 ... 0.34670913 0.1202205  0.15453582]
 [0.10087176 0.0325541  0.06634608 ... 0.13564515 0.10328979 0.28718388]
 ...
 [0.09735985 0.03655784 0.02110462 ... 0.00878494 0.02641206 0.09358978]
 [0.06596183 0.02479467 0.01463192 ... 0.01295443 0.01894925 0.1366848 ]
 [0.07484622 0.02483043 0.00924662 ... 0.00521394 0.03489915 0.15958416]]


0.0

-------------

In [132]:
y_pred_expr_test,y_pred_va_test,y_pred_aus_test=y_pred_expr,y_pred_va,y_pred_aus

In [155]:
y_pred_expr_test_1,y_pred_va_test_1,y_pred_aus_test_1=y_pred_expr,y_pred_va,y_pred_aus

In [167]:
print('\nVA')
w_blend=0.8
y_pred_va_blend=w_blend*y_pred_va_test+(1-w_blend)*y_pred_va_test_1
pred_V=y_pred_va_blend[:,0]
pred_A=y_pred_va_blend[:,1]
print('VA',pred_V.shape,pred_A.shape,pred_V[:5],pred_A[:5])


print('\nExpression')
w_blend=0.5
y_pred_expr_blend=w_blend*y_pred_expr_test+(1-w_blend)*y_pred_expr_test_1
y_pred=np.argmax(y_pred_expr_blend,axis=1)
print('Expression',y_pred_expr_blend.shape,y_pred.shape,y_pred[:5])


print('\nAUs')
w_blend=0.5
y_pred_aus_blend=w_blend*y_pred_aus_test+(1-w_blend)*y_pred_aus_test_1
thresholds=0.5
aus_pred = ((y_pred_aus_blend >= thresholds) * 1)
print('AUs',y_pred_aus_blend.shape,aus_pred.shape,aus_pred[:5])


VA
VA (51159,) (51159,) [0.5986318  0.6083133  0.26851386 0.10634097 0.06756733] [0.54123175 0.43009633 0.51555115 0.5765881  0.5908811 ]

Expression
Expression (51159, 8) (51159,) [4 4 7 7 7]

AUs
AUs (51159, 12) (51159, 12) [[0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]]


In [168]:
#with open(os.path.join(resdir,'5_mtl_ddamfn_enet0_aus_0.5.txt'), 'w') as f:
with open(os.path.join(resdir,'11_mtl_ddamfn_enet0_train_val_aus_0.5.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)

In [169]:
print('\nVA')
w_blend=0.8
y_pred_va_blend=w_blend*y_pred_va_test+(1-w_blend)*y_pred_va_test_1
num_y=len(y_pred_va_blend)
sigma,delta=100000,10
y_pred_va_smooth=np.zeros_like(y_pred_va_blend)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_test[j][0]==videoname_frames_test[i][0]:
            k=math.exp(-(videoname_frames_test[j][1]-videoname_frames_test[i][1])**2/sigma)
            y_pred_va_smooth[i]+=k*y_pred_va_blend[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_test[j][0]==videoname_frames_test[i][0]:
            k=math.exp(-(videoname_frames_test[j][1]-videoname_frames_test[i][1])**2/sigma)
            y_pred_va_smooth[i]+=k*y_pred_va_blend[j]
            k_sum+=k
        else:
            break
    y_pred_va_smooth[i]/=k_sum

pred_V=y_pred_va_smooth[:,0]
pred_A=y_pred_va_smooth[:,1]
print('VA',pred_V.shape,pred_A.shape,pred_V[:5],pred_A[:5])

print('\nExpression')
w_blend=0.5
y_pred_expr_blend=w_blend*y_pred_expr_test+(1-w_blend)*y_pred_expr_test_1
num_y=len(y_pred_expr_blend)
sigma,delta=100000,50
y_pred_expr_smooth=np.zeros_like(y_pred_expr_blend)
for i in range(num_y):
    start,end=max(0,i-delta),min(num_y,i+delta+1)
    k_sum=0

    for j in range(i-1,start,-1):
        if videoname_frames_test[j][0]==videoname_frames_test[i][0]:
            k=math.exp(-(videoname_frames_test[j][1]-videoname_frames_test[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr_blend[j]
            k_sum+=k
        else:
            break
    for j in range(i,end):
        if videoname_frames_test[j][0]==videoname_frames_test[i][0]:
            k=math.exp(-(videoname_frames_test[j][1]-videoname_frames_test[i][1])**2/sigma)
            y_pred_expr_smooth[i]+=k*y_pred_expr_blend[j]
            k_sum+=k
        else:
            break
    y_pred_expr_smooth[i]/=k_sum

y_pred=np.argmax(y_pred_expr_smooth,axis=1)
print('Expression',y_pred_expr_smooth.shape,y_pred.shape,y_pred[:5])


print('\nAUs')
w_blend=0.5
y_pred_aus_blend=w_blend*y_pred_aus_test+(1-w_blend)*y_pred_aus_test_1
thresholds=0.5
aus_pred = ((y_pred_aus_blend >= thresholds) * 1)
print('AUs',y_pred_aus_blend.shape,aus_pred.shape,aus_pred[:5])


VA
VA (51159,) (51159,) [0.43097115 0.44984818 0.47526142 0.49827132 0.5277503 ] [0.5578462  0.56265193 0.56484723 0.5671396  0.56592816]

Expression
Expression (51159, 8) (51159,) [4 4 4 4 4]

AUs
AUs (51159, 12) (51159, 12) [[0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]]


In [170]:
#with open(os.path.join(resdir,'6_mtl_ddamfn_enet0_smooth_aus_0.5.txt'), 'w') as f:
with open(os.path.join(resdir,'11_mtl_ddamfn_enet0_train_val_smooth_aus_0.5.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)